In [151]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import PIL
from PIL import Image, ImageDraw

import random
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqn
%matplotlib inline

from dataset import B, V, F, R, P, action, ImagesBatch
from dataset.models.tf import TFModel
from dataset.models.tf.layers import conv_block
from dataset.batch_image import transform_actions
import mnist

from dataset import DatasetIndex, Dataset, Batch

In [ ]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [ ]:
# 2 classes: four and others

four=[]
others=[]
for i in range(0, len(train_labels)):
    if train_labels[i]==4:
        four.append(i)
    else:
        others.append(i)
random.shuffle(others)

In [147]:
# new_mage_size 56 x 28
train_4 = train_images[four[0]]
train_4 = np.append(train_4, train_images[others[0]],axis=0)
zipped = zip(four, others[0:len(four)])
for i,j in zipped:
    if i != four[0]:
        train_4 = np.append(train_4, train_images[i], axis=0)
        train_4 = np.append(train_4, train_images[j], axis=0)
train_4 = train_4.reshape((len(four),56,28))
y_4 = np.ones((len(four)))

In [148]:
train_others = train_images[others[len(four)]]
for i in others[len(four):len(four)*3]:
    if i != others[len(four)]:
        train_others = np.append(train_others, train_images[i], axis=0)
train_others = train_others.reshape((len(four),56,28))
y_others = np.zeros((len(four)))

In [ ]:
train_images_new = np.concatenate((train_4, train_others), axis=0)
train_labels_new = np.concatenate((y_4, y_others),axis=0)

In [ ]:
class MyBatch(Batch):
    components = 'images', 'labels'

In [ ]:
client_index = list(range(train_labels_new.shape[0]))
client_dataset = Dataset(client_index, batch_class=MyBatch, preloaded=(train_images_new, train_labels_new))
client_dataset.cv_split([.8, .1, .1])

In [ ]:
class MyModel(TFModel):
    @classmethod
    def default_config(cls):
        config = TFModel.default_config()
        config['body'].update(dict(filters=[2], kernel_size=3, layout='cna'))
        config['head'].update(dict(units=2, layout='df', dropout_rate=.2))
        return config

    def build_config(self, names=None):
        config = super().build_config(names)
        config['head']['units'] = self.num_classes('targets')
        config['head']['filters'] = self.num_classes('targets')
        return config

    @classmethod
    def body(cls, inputs, name='body', **kwargs):
        kwargs = cls.fill_params('body', **kwargs)
        x = conv_block(inputs, **kwargs)
        return x

In [ ]:
inputs_config = {
    'images': {'shape': [56, 28]},
    'labels': {'shape': 1, 'name': 'targets'}
}

In [ ]:
config = {
    'inputs': dict(images={'shape': B('image_shape')},
                                                   labels={'classes': 2, 'transform': 'ohe', 'name': 'targets'}),
    'loss': 'mse',
    'optimizer': {'name':'GradientDescentOptimizer', 'learning_rate': 0.01},
    'body/units': 1,
}

In [ ]:
train_pipeline = ( 
    client_dataset.train.p
    .init_variable('loss_history', init_on_each_run=list)
    .init_model('dynamic', MyModel, 'my_model', config=config)
    .train_model('my_model', fetches='loss',
                 feed_dict={'images':B('images'),
                            'labels': B('labels')},save_to=V('loss_history'), mode='a')
)

In [ ]:
BATCH_SIZE = 64
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=True, bar=True, prefetch=1)